In [ ]:
import pandas as pd
import numpy as np

I definitely recommend people exploring the dataset with BigQuery, esp following the setup described by Brian. It's just my internet connection is not so stable to access google products right now. But the code below can be easily transferred to, say colab, by just changing how you load the tables into pandas.

# MIMIC-III

Here I tried to look at mimic-iii and see if i can reproduce the numbers in the literature, like the `Lin et al, 2019` paper.

In [ ]:
stays = pd.read_csv('ICUSTAYS.csv.gz')

In [ ]:
stays.head(2)

In [ ]:
counts = stays.groupby(['HADM_ID']).size().reset_index(name='COUNTS')
def merge_stays_counts(table1, table2):
    return table1.merge(table2, how='inner', left_on=['HADM_ID'], right_on=['HADM_ID'])
stays = merge_stays_counts(stays, counts)

In [ ]:
max_outtime = stays.groupby(['HADM_ID'])['OUTTIME'].transform(max) == stays['OUTTIME']
stays['MAX_OUTTIME'] = max_outtime.astype(int)

In [ ]:
transferback = (stays.COUNTS > 1) & (stays.MAX_OUTTIME == 0)
stays[transferback]

Cool. 3746 seems to be a good number here. MIMIC-III is working alright for us.

In [ ]:
stays[stays.HADM_ID == 127889][['HADM_ID', 'ICUSTAY_ID', 'INTIME', 'OUTTIME']]

Notice the interval between `OUTTIME` and the subsequent `INTIME`, because they will disappear in MIMIC-IV...

# MIMIC-IV

Alright, let's look at the equivalent table in mimic-iv, and follow the procedures we took with mimic-iii to get the readmission counts.

In [ ]:
icu = pd.read_csv('../MIMIC-IV/icustays.csv.gz')
icu.head(2)

In [ ]:
max_out=icu.groupby(['hadm_id'])['outtime'].transform(max) == icu['outtime']
icu['MAXOUT'] = max_out.astype(int)

In [ ]:
counts = icu.groupby(['hadm_id']).size().reset_index(name='COUNTS')
icu = icu.merge(counts, on=['hadm_id']).sort_values(by=['hadm_id','intime'])

In [ ]:
mul = icu[ (icu.COUNTS>1)]
mul[ (icu.MAXOUT==0)]

In [ ]:
mul[mul.hadm_id == 22415651]

Above is just repeat what we did with mimic-iii. But some weird things happen, that the interval between `outtime` and next `intime` is gone, which raise the question on if the pt was directly transfer from one ICU unit to another. In this case, are they also readmission? So I'm not sure if 4644 is the right number. We need to look at rows in the derived `mul` table we have to see if such is the case: _every pt who visited ICU at least twice is directly transfered from one to the other._

In [ ]:
# here I shift the outtime to compare with subsequent intime
shift = mul['outtime'].shift(1)
# I create a mask to locate first icu visit
shift[mul.MAXOUT==0] = np.NaN
mul['shift_outtime']=shift

In [ ]:
# let's see how it goes with the whole table
msk = mul['shift_outtime'] != mul['intime']
mul[msk & ~mul['shift_outtime'].isna()]

Bummer! It looks like those intervals in mimic-iii totally disappeared in the `icustays` table in mimic-iv. Notice that `icustays` itself is a derived table from `transfers`, so there could be some change in how the table is derived between iii and iv. 

In [ ]:
mul[mul.hadm_id == 22415651]

So let's look at transfers to see if we can ge what we want.

In [ ]:
tran = pd.read_csv('../MIMIC-IV/transfers.csv.gz')
tran[tran.hadm_id == 22415651].sort_values(by=['intime']) # take a look at an example

In [ ]:
# transfers contains a lot of information and is a bit messy, here I just filter it a bit
tran = tran[['hadm_id', 'eventtype', 'careunit', 'intime', 'outtime']]
tran = tran[(tran['eventtype'] == 'transfer') | (tran['eventtype'] == 'admit')]
tran

In [ ]:
# here create a table with only icu transfers
icumsk = tran['careunit'].fillna('none').apply(lambda x: 1 if 'ICU' in x else 0) 
tran_icu = tran[icumsk==1]

In [ ]:
# then identify hadm with multiple icu transfers
tran_ct = tran_icu.groupby('hadm_id').size().reset_index(name='COUNTS')
tran_mul = tran_icu.merge(tran_ct, on=['hadm_id'])
tran_mul = tran_mul[tran_mul.COUNTS > 1].sort_values(by=['hadm_id', 'intime'])

In [ ]:
tran_mul.hadm_id.value_counts(), len(tran_mul)

In [ ]:
# then calculate the time interval bewteen different icu transfers, again using shifted intime and outtime
tran_mul.intime = pd.to_datetime(tran_mul.intime)
tran_mul.outtime = pd.to_datetime(tran_mul.outtime)

tran_mul['diff_hour'] =  (tran_mul.intime.shift(-1) - tran_mul.outtime) / pd.Timedelta('1 hour')
tran_mul

In [ ]:
# finally, let's filter with 72hr
final = tran_mul[(tran_mul.diff_hour < 72) & (tran_mul.diff_hour > 0 )]
final['dur_hour'] = (final.outtime - final.intime) / pd.Timedelta('1 hour')
final

In [ ]:
final.hadm_id.value_counts()

Now I get the numbers in slack: 6586 (total icu readmins, including duplicates with multiple readmin, at most 4 in some cases) and 5910 (count of hospital stays or patients). Changing the minimum `diff_hour` from 0 to 1 is where the final 3946 (hadms) comes from, shown below.

In [ ]:
final = tran_mul[(tran_mul.diff_hour < 72) & (tran_mul.diff_hour > 1 )]
final

In [ ]:
final.hadm_id.value_counts()

 without having the patient spending time at other wards. I wonder how those stays are grouped. I found a case (`hadm_id` =22415651) with a pretty lengthy stay (`stay_id`= 35797310), but the `transfer` table seems to show t

# eICU

Alright, finally, eICU. The structure here is totally different from the previous two mimic datasets. It turns out they have a patient table to record hospital course that is pretty handy for us to work with. Here we go

In [ ]:
pt = pd.read_csv('../patient.csv.gz') # just the eICU table downloaded from physionet
pt.head(3)

In [ ]:
# it's a big table, but we only need a handful of columns
eicu_cols = ['patientunitstayid', 'patienthealthsystemstayid', 'hospitaladmitoffset','unitdischargeoffset']
unit = pt[eicu_cols]
unit

In [ ]:
# as before, let's focus on stayid with multiple unitstayid
count = unit.groupby('patienthealthsystemstayid').size().reset_index(name='counts')
unit = unit.merge(count, how='left')

In [ ]:
mul = unit[unit.counts>1].sort_values(by=['patienthealthsystemstayid', 'hospitaladmitoffset'], ascending=False)
mul

In [ ]:
def create_shifted_offset(tmp):
    # use shiftid to identify first or pseudo second icu stay
    tmp['shiftid'] = tmp.patienthealthsystemstayid.shift(1)
    
    tmp['firsticu'] = tmp.patienthealthsystemstayid != tmp.shiftid
    tmp['firsticu'] = tmp['firsticu'].astype(int)
    
    # get the discharge time based on illustration in https://eicu-crd.mit.edu/eicutables/patient/
    tmp['unitdisch'] = tmp.hospitaladmitoffset - tmp.unitdischargeoffset
    tmp['shiftunitdisch'] = tmp['unitdisch'].shift(1) # shift it to examine either the second icu stay is a pseudo one
    # by pseudo, i'm referring to those visits with no interval between intime and prev outtime
    
    msk1 = (tmp.shiftunitdisch != tmp.hospitaladmitoffset )
    msk2 = (tmp.firsticu ==0) # true second icu visit
    tmp = tmp[msk1 & msk2]
    
    tmp['diff_time'] = tmp.shiftunitdisch - tmp.hospitaladmitoffset  # get interval between two icu stays
    
    return tmp

In [ ]:
mul_icus = create_shifted_offset(mul)

In [ ]:
final = mul_icus[(mul_icus.diff_time < 60*72) & (mul_icus.diff_time > 0)]

In [ ]:
final.patienthealthsystemstayid.value_counts()

In [ ]:
pt.patienthealthsystemstayid.value_counts()

In [ ]:
num_pt_w_readmin = len(final.patienthealthsystemstayid.unique()) 
num_total_pt = len(pt.patienthealthsystemstayid.unique())
num_pt_w_readmin, num_total_pt, num_pt_w_readmin/num_total_pt